# Vamos a leer el cluster 2 y generar una predicción basado en los trabajos de Diana

vamos por paso
Vamos a ver si por memoria tengo que partir

En este caso no voy a generar las covariables que generó Daiana

Voy a probar que corra, si corre luego le agrego las variables y veo los cambios en Kaggle




## Preparo todo

In [ ]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [ ]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend

import itertools

import gc
from datetime import datetime

#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

In [ ]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import matplotlib.pyplot as plt



### funciones Di

In [ ]:
def percentage_safe(numerator: pd.Series, denominator: pd.Series, dtype='float32', fillna=None) -> pd.Series:
    """
    Calcula un porcentaje seguro como numerator / denominator.
    - Reemplaza divisiones por cero o NaN con NaN.
    - Opcionalmente convierte a float32.
    - Puede rellenar NaNs con `fillna`.
    """
    result = (numerator / denominator).mask((denominator == 0) | (denominator.isna()))
    if fillna is not None:
        result = result.fillna(fillna)
    return result.astype(dtype)

In [ ]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

In [ ]:
def clasificar_estado_producto(df: pd.DataFrame) -> None:
    """
    Clasifica el estado de los productos en un DataFrame basado en su ciclo de vida y ventas.

    La función evalúa una serie de condiciones y crea una nueva columna 'estado_producto'
    en el DataFrame proporcionado. La modificación se hace 'inplace'.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos de los productos.
                           Debe incluir las columnas:
                           - 'meses_vida_producto'
                           - 'producto_crecimiento_ventas_suavizado'
                           - 'share_producto_en_categoria_suavizado'

    Returns:
        None: La función modifica el DataFrame directamente.
    """
    # Lista de condiciones en el orden de prioridad requerido
    condiciones = [
        (df['meses_vida_producto'] >= 0)& (df['meses_vida_producto'] <= 3),
        (df['producto_crecimiento_ventas_suavizado'] == 0),
        (df['producto_crecimiento_ventas_suavizado'] > 0.1) & (df['share_producto_en_categoria_suavizado'] > 0.1),
        (df['producto_crecimiento_ventas_suavizado'] < -0.1) & (df['share_producto_en_categoria_suavizado'] < -0.1)
    ]

    # Lista de resultados correspondientes a cada condición
    resultados = [
        'inicial',
        'sin ventas',
        'crecimiento',
        'contraccion'
    ]

    # El valor por defecto si ninguna condición se cumple es 'estable'
    df['estado_producto'] = np.select(condiciones, resultados, default='estable')

In [ ]:
def calcular_ventanas_moviles(df, campo_base, prefijo_lag, tipo_agregacion, nombre_sufijo=None):
    """
    Calcula estadísticas móviles sobre ventanas específicas (3, 6 y 12 periodos).

    Parámetros:
    - df: DataFrame.
    - campo_base: nombre de la columna base (ej. "cat_total_tn").
    - prefijo_lag: parte de la cadena que compone la columna (ej. "_ma_").
    - tipo_agregacion: 'mean', 'std', 'min', 'max', 'median'.
    - nombre_sufijo: sufijo opcional para el nombre de la columna resultante.
    """
    if tipo_agregacion not in {"mean", "std", "min", "max", "median"}:
        raise ValueError("Agregación no soportada.")

    ventanas = [3, 6, 12]
    for ventana in ventanas:
        # Se genera el nombre completo de la columna que ya existe en el DataFrame.
        columna_ventana = f"{campo_base}{prefijo_lag}{ventana}"
        nombre_col = f"{campo_base}_{tipo_agregacion}_movil_{ventana}"
        if nombre_sufijo:
            nombre_col += f"_{nombre_sufijo}"
        # Aquí se aplica la agregación; en realidad, dado que se opera sobre una sola columna,
        # esto simplemente devolverá el mismo valor (o podría servir para dar formato).
        df[nombre_col] = getattr(df[[columna_ventana]], tipo_agregacion)(axis=1)

## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [ ]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_3 = pd.read_parquet(DATOS_DIR +'df_cluster_3.parquet')
#df_cluster_3.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [ ]:
# me quedo con las filas de df_cluster_3 cuyo product_id esté en productos_validar
###################################################
##### OJO, en realidad debo dejar todo
##### dejo las otras series que ayuden
##### saco de todo el conjunto lo que necesito al final
#############################
#df_cluster_X = df_cluster_3[df_cluster_3['product_id'].isin(productos_validar['product_id'])]
df_cluster_X = df_cluster_3.copy()
df_cluster_X.shape

(174785, 45)

In [ ]:
df_cluster_X.shape

(174785, 45)

In [ ]:
# tengo que pasar a timestamp todas las columnas con formato period[M] de df_cluster_X
# Identificamos automáticamente las columnas de tipo Period[M]
period_cols = df_cluster_X.select_dtypes(include=['period[M]']).columns

# Convertimos todas esas columnas a datetime usando .dt.to_timestamp()
df_cluster_X[period_cols] = df_cluster_X[period_cols].apply(lambda col: col.dt.to_timestamp())
df_cluster_X.dtypes

,0
customer_id,int64
product_id,int64
date_mes,datetime64[ns]
first_customer_date,datetime64[ns]
last_customer_date,datetime64[ns]
first_product_date,datetime64[ns]
last_product_date,datetime64[ns]
tn,float64
dif_tn,float64
dif_por,float64


In [ ]:
## A partir de df_cluster_X quiero generar un df_historia que tenga indice, primera columna produc_cust_id segunda columna el mínimo valor distinto de cero para dicho product_cust_id
# Filtra las filas donde tn es distinto de cero
df_filtrado = df_cluster_X[df_cluster_X['tn'] != 0]

# Agrupa por product_cust_id y calcula el mínimo de tn (distinto de cero)
df_historia = (
    df_filtrado
    .groupby('product_cust_id', as_index=False)['tn']
    .min()
    .rename(columns={'tn': 'min'})
)

print(df_historia.head())

  product_cust_id     min
0     20001_10003   1.848
1     20001_10007  28.934
2     20001_10013  16.019
3     20001_10040   0.123
4     20001_10053   0.056


## agrando el df con lo realizado por Daiana / CHAT / di

In [ ]:
# hacemos limpieza
del df_cluster_3
gc.collect()

106

In [ ]:
df_cluster_X.columns

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacén', 'Panadería', 'Lácteos', 'Carnes',
       'Verdulería y frutería', 'Artículos de limpieza y perfumería',
       'Indumentaria, calzado y textiles para el hogar',
       'Electrónicos y artículos para el hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id'],
      dtype='object')

#### le seguimos la corriente a Di, en algunos puntos

In [ ]:
df_cluster_X.shape

(174785, 45)

In [ ]:
# Asumimos df_cluster_X con columnas: product_id, customer_id, date_mes (tipo datetime o Period), tn

# ---------- PASO 1: Agregar tn total por customer_id + date_mes
customer_monthly = (
    df_cluster_X.groupby(['customer_id', 'date_mes'], as_index=False)
    ['tn'].sum()
    .rename(columns={'tn': 'tn_sum_cus'})
)

# ---------- PASO 2: Agregar tn total por product_id + date_mes
product_monthly = (
    df_cluster_X.groupby(['product_id', 'date_mes'], as_index=False)
    ['tn'].sum()
    .rename(columns={'tn': 'tn_sum_prod'})
)

def calcular_estadisticas_adicionales(df, group_col, base_col):
    """
    En DataFrame con columnas (group_col, date_mes, base_col) calcula:
    - rolling mean (3,6,9,12)
    - lags (3,6,9,12)
    - deltas (con rolling y lag)
    - para cada métrica mensual base (tn_sum_*), rolling y lag adiciona min, max y std
    """
    ventana_list = [3,6,9,12]
    df = df.sort_values([group_col, 'date_mes']).copy()

    # Medias móviles rolling mean
    for ventana in ventana_list:
        col_rolling = f'{base_col}_rolling_{ventana}m'
        df[col_rolling] = (
            df.groupby(group_col)[base_col]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).mean())
        )

        # estadisticas rolling para las medias móviles
        df[f'{col_rolling}_min'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).min())
        )
        df[f'{col_rolling}_max'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).max())
        )
        df[f'{col_rolling}_std'] = (
            df.groupby(group_col)[col_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).std())
        )

    # Lags
    for lag in ventana_list:
        col_lag = f'{base_col}_lag_{lag}m'
        df[col_lag] = df.groupby(group_col)[base_col].shift(lag)

        # estadisticas para lags en ventana moviles
        df[f'{col_lag}_min'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).min())
        )
        df[f'{col_lag}_max'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).max())
        )
        df[f'{col_lag}_std'] = (
            df.groupby(group_col)[col_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).std())
        )

    # Deltas (con rolling y lag)
    for ventana in ventana_list:
        col_rolling = f'{base_col}_rolling_{ventana}m'
        col_delta_rolling = f'delta_rolling_{ventana}m'
        df[col_delta_rolling] = df[base_col] - df[col_rolling]

        # estadisticas para deltas rolling
        df[f'{col_delta_rolling}_min'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).min())
        )
        df[f'{col_delta_rolling}_max'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).max())
        )
        df[f'{col_delta_rolling}_std'] = (
            df.groupby(group_col)[col_delta_rolling]
            .transform(lambda x: x.rolling(window=ventana, min_periods=1).std())
        )

    for lag in ventana_list:
        col_lag = f'{base_col}_lag_{lag}m'
        col_delta_lag = f'delta_lag_{lag}m'
        df[col_delta_lag] = df[base_col] - df[col_lag]

        # estadisticas para deltas lag
        df[f'{col_delta_lag}_min'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).min())
        )
        df[f'{col_delta_lag}_max'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).max())
        )
        df[f'{col_delta_lag}_std'] = (
            df.groupby(group_col)[col_delta_lag]
            .transform(lambda x: x.rolling(window=lag, min_periods=1).std())
        )

    # También agregamos para base_col sin transformaciones estadísticas básicas por grupo y mes
    df[f'{base_col}_min'] = df.groupby(group_col)[base_col].transform('min')
    df[f'{base_col}_max'] = df.groupby(group_col)[base_col].transform('max')
    df[f'{base_col}_std'] = df.groupby(group_col)[base_col].transform('std')

    return df

# ---------- PASO 3: Calcular para customer_monthly
customer_monthly = calcular_estadisticas_adicionales(customer_monthly, 'customer_id', 'tn_sum_cus')

# ---------- PASO 4: Calcular para product_monthly
product_monthly = calcular_estadisticas_adicionales(product_monthly, 'product_id', 'tn_sum_prod')

# ---------- PASO 5: Unir métricas al DataFrame original

# Primero unimos tn_sum_cus + métricas customer
df_cluster_X = df_cluster_X.merge(
    customer_monthly,
    on=['customer_id', 'date_mes'],
    how='left',
    suffixes=('', '_cus')
)

# Luego unimos tn_sum_prod + métricas product
df_cluster_X = df_cluster_X.merge(
    product_monthly,
    on=['product_id', 'date_mes'],
    how='left',
    suffixes=('', '_prod')
)

# ---------- PASO 6: El DataFrame df_cluster_X tiene todas las métricas y estadísticas solicitadas
print(df_cluster_X.columns)

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por',
       ...
       'delta_lag_9m_min_prod', 'delta_lag_9m_max_prod',
       'delta_lag_9m_std_prod', 'delta_lag_12m_prod', 'delta_lag_12m_min_prod',
       'delta_lag_12m_max_prod', 'delta_lag_12m_std_prod', 'tn_sum_prod_min',
       'tn_sum_prod_max', 'tn_sum_prod_std'],
      dtype='object', length=181)


In [ ]:
df_cluster_X.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id,tn_sum_cus,tn_sum_cus_rolling_3m,tn_sum_cus_rolling_3m_min,tn_sum_cus_rolling_3m_max,tn_sum_cus_rolling_3m_std,tn_sum_cus_rolling_6m,tn_sum_cus_rolling_6m_min,tn_sum_cus_rolling_6m_max,tn_sum_cus_rolling_6m_std,tn_sum_cus_rolling_9m,tn_sum_cus_rolling_9m_min,tn_sum_cus_rolling_9m_max,tn_sum_cus_rolling_9m_std,tn_sum_cus_rolling_12m,tn_sum_cus_rolling_12m_min,tn_sum_cus_rolling_12m_max,tn_sum_cus_rolling_12m_std,tn_sum_cus_lag_3m,tn_sum_cus_lag_3m_min,tn_sum_cus_lag_3m_max,tn_sum_cus_lag_3m_std,tn_sum_cus_lag_6m,tn_sum_cus_lag_6m_min,tn_sum_cus_lag_6m_max,tn_sum_cus_lag_6m_std,tn_sum_cus_lag_9m,tn_sum_cus_lag_9m_min,tn_sum_cus_lag_9m_max,tn_sum_cus_lag_9m_std,tn_sum_cus_lag_12m,tn_sum_cus_lag_12m_min,tn_sum_cus_lag_12m_max,tn_sum_cus_lag_12m_std,delta_rolling_3m,delta_rolling_3m_min,delta_rolling_3m_max,delta_rolling_3m_std,delta_rolling_6m,delta_rolling_6m_min,delta_rolling_6m_max,delta_rolling_6m_std,delta_rolling_9m,delta_rolling_9m_min,delta_rolling_9m_max,delta_rolling_9m_std,delta_rolling_12m,delta_rolling_12m_min,delta_rolling_12m_max,delta_rolling_12m_std,delta_lag_3m,delta_lag_3m_min,delta_lag_3m_max,delta_lag_3m_std,delta_lag_6m,delta_lag_6m_min,delta_lag_6m_max,delta_lag_6m_std,delta_lag_9m,delta_lag_9m_min,delta_lag_9m_max,delta_lag_9m_std,delta_lag_12m,delta_lag_12m_min,delta_lag_12m_max,delta_lag_12m_std,tn_sum_cus_min,tn_sum_cus_max,tn_sum_cus_std,tn_sum_prod,tn_sum_prod_rolling_3m,tn_sum_prod_rolling_3m_min,tn_sum_prod_rolling_3m_max,tn_sum_prod_rolling_3m_std,tn_sum_prod_rolling_6m,tn_sum_prod_rolling_6m_min,tn_sum_prod_rolling_6m_max,tn_sum_prod_rolling_6m_std,tn_sum_prod_rolling_9m,tn_sum_prod_rolling_9m_min,tn_sum_prod_rolling_9m_max,tn_sum_prod_rolling_9m_std,tn_sum_prod_rolling_12m,tn_sum_prod_rolling_12m_min,tn_sum_prod_rolling_12m_max,tn_sum_prod_rolling_12m_std,tn_sum_prod_lag_3m,tn_sum_prod_lag_3m_min,tn_sum_prod_lag_3m_max,tn_sum_prod_lag_3m_std,tn_sum_prod_lag_6m,tn_sum_prod_lag_6m_min,tn_sum_prod_lag_6m_max,tn_sum_prod_lag_6m_std,tn_sum_prod_lag_9m,tn_sum_prod_lag_9m_min,tn_sum_prod_lag_9m_max,tn_sum_prod_lag_9m_std,tn_sum_prod_lag_12m,tn_sum_prod_lag_12m_min,tn_sum_prod_lag_12m_max,tn_sum_prod_lag_12m_std,delta_rolling_3m_prod,delta_rolling_3m_min_prod,delta_rolling_3m_max_prod,delta_rolling_3m_std_prod,delta_rolling_6m_prod,delta_rolling_6m_min_prod,delta_rolling_6m_max_prod,delta_rolling_6m_std_prod,delta_rolling_9m_prod,delta_rolling_9m_min_prod,delta_rolling_9m_max_prod,delta_rolling_9m_std_prod,delta_rolling_12m_prod,delta_rolling_12m_min_prod,delta_rolling_12m_max_prod,delta_rolling_12m_std_prod,delta_lag_3m_prod,delta_lag_3m_min_prod,delta_lag_3m_max_prod,delta_lag_3m_std_prod,delta_lag_6m_prod,delta_lag_6m_min_prod,delta_lag_6m_max_prod,delta_lag_6m_std_prod,delta_lag_9m_prod,delta_lag_9m_min_prod,delta_lag_9m_max_prod,delta_lag_9m_std_prod,delta_lag_12m_prod,delta_lag_12m_min_prod,delta_lag_12m_max_prod,delta_lag_12m_std_prod,tn_sum_prod_min,tn_sum_prod_max,tn_sum_prod_std
0,10001,20003,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,100.213,0.0,0.0,4,4,4,0,60,2017,1,1,1,FOODS,FOODS_ADEREZOS,FOODS_ADEREZOS_Mayonesa,FOODS_ADEREZOS_Mayonesa_NATURA,FOODS_ADEREZOS_Mayonesa_NATURA_475.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47,20003_10001,972.227,972.227,972.227,972.227,NaN,972.227,972.227,972.22

In [ ]:
df_cluster_X.shape

(174785, 181)

In [ ]:
# Aseguramos que df_cluster_X contiene product_cus_id columna única para cada product_id + customer_id
# Vamos a trabajar agrupando por product_cus_id

df_cluster_X = df_cluster_X.sort_values(['product_cust_id', 'date_mes']).copy()

def feature_engineering_product_cus(df, group_col, target_col):
    ventana_list = [3,6,9,12]
    max_lag = 12

    df = df.copy()

    # Lags de 1 a 12 meses
    for lag in range(1, max_lag+1):
        col_lag = f'{target_col}_lag{lag}'
        df[col_lag] = df.groupby(group_col)[target_col].shift(lag)

    # Función para calcular pendiente (tendencia)
    def calc_tendencia(x):
        if x.dropna().shape[0] < 2:
            return np.nan
        X = np.arange(len(x)).reshape(-1,1)
        y = x.values.reshape(-1,1)
        mask = ~np.isnan(y).flatten()
        if mask.sum() < 2:
            return np.nan
        lr = LinearRegression()
        lr.fit(X[mask], y[mask])
        return lr.coef_[0][0]

    for ventana in ventana_list:
        # rolling mean, min, max, std
        df[f'{target_col}_mean_{ventana}m'] = (
            df.groupby(group_col)[target_col]
            .transform(lambda x: x.rolling(ventana, min_periods=1).mean())
        )
        df[f'{target_col}_min_{ventana}m'] = (
            df.groupby(group_col)[target_col]
            .transform(lambda x: x.rolling(ventana, min_periods=1).min())
        )
        df[f'{target_col}_max_{ventana}m'] = (
            df.groupby(group_col)[target_col]
            .transform(lambda x: x.rolling(ventana, min_periods=1).max())
        )
        df[f'{target_col}_std_{ventana}m'] = (
            df.groupby(group_col)[target_col]
            .transform(lambda x: x.rolling(ventana, min_periods=1).std())
        )

        # tendencia (pendiente)
        df[f'{target_col}_trend_{ventana}m'] = (
            df.groupby(group_col)[target_col]
            .transform(lambda x: x.rolling(ventana, min_periods=2).apply(calc_tendencia, raw=False))
        )

        # meses desde máximo en ventana
        def meses_desde_max_series(x):
            if x.isnull().all():
                return np.nan
            idx_max = x.idxmax()
            idx_actual = x.index[-1]
            return (idx_actual - idx_max)

        df[f'{target_col}_meses_desde_max_{ventana}m'] = df.groupby(group_col)[target_col].transform(
            lambda x: x.rolling(ventana, min_periods=1).apply(meses_desde_max_series, raw=False)
        )

    # Limpiamos métricas para filas donde tn es NaN (producto-cliente no activo mes)
    cols_nuevas = [col for col in df.columns if col.startswith(f'{target_col}_') and col != target_col]
    for col in cols_nuevas:
        df.loc[df[target_col].isna(), col] = np.nan

    return df

# Aplicamos función pasando 'product_cus_id' como agrupador y 'tn' como la métrica sobre la que trabajar
df_cluster_X = feature_engineering_product_cus(df_cluster_X, 'product_cust_id', 'tn')

print(df_cluster_X.head())

      customer_id  product_id   date_mes first_customer_date  \
9135        10003       20001 2017-01-01          2017-01-01   
9136        10003       20001 2017-02-01          2017-01-01   
9137        10003       20001 2017-03-01          2017-01-01   
9138        10003       20001 2017-04-01          2017-01-01   
9139        10003       20001 2017-05-01          2017-01-01   

     last_customer_date first_product_date last_product_date       tn  dif_tn  \
9135         2019-12-01         2017-01-01        2019-12-01  143.494   0.000   
9136         2019-12-01         2017-01-01        2019-12-01   20.483   0.000   
9137         2019-12-01         2017-01-01        2019-12-01  137.875  -0.056   
9138         2019-12-01         2017-01-01        2019-12-01   68.893 -11.215   
9139         2019-12-01         2017-01-01        2019-12-01  135.122 -14.999   

        dif_por  vida_prod  vida_customer  meses_on  meses_off  vida_restante  \
9135  0.000e+00          4              4      

In [ ]:
df_cluster_X.shape

(174785, 217)

In [ ]:
# Lista de columnas de categoría según lo indicado
categorias = [
    'cat1',
    'cat1_cat2',
    'cat1_cat2_cat3',
    'cat1_cat2_cat3_brand',
    'cat1_cat2_cat3_brand_size'
]

# Ordenamos para orden temporal y por cliente
df_cluster_X = df_cluster_X.sort_values(['customer_id', 'date_mes']).copy()

# ---------- PASO 1: ventas totales tn por cliente y mes para cada categoría ----------

# Lista para almacenar dataframes de cada categoría
ventas_cat_cus_mes_list = []

for cat in categorias:
    # Seleccionar solo filas donde la categoría está presente y no nula
    df_cat = df_cluster_X[df_cluster_X[cat].notnull()]

    if df_cat.empty:
        continue

    # Agrupar venta tn total por cliente y mes para esa categoría
    ventas_cat = (
        df_cat.groupby(['customer_id', 'date_mes'])['tn']
        .sum()
        .reset_index()
        .rename(columns={'tn': f'tn_{cat}_cus_mes'})
    )

    ventas_cat_cus_mes_list.append(ventas_cat)

# Unir todos en un solo DataFrame por customer_id y date_mes, usando outer join
ventas_cat_cus_mes_all = ventas_cat_cus_mes_list[0] if len(ventas_cat_cus_mes_list) > 0 else pd.DataFrame()

for df_cat in ventas_cat_cus_mes_list[1:]:
    ventas_cat_cus_mes_all = ventas_cat_cus_mes_all.merge(
        df_cat,
        on=['customer_id', 'date_mes'],
        how='outer'
    )

# Reemplazamos NaN por 0 para ventas donde no hubo ocurrencia
ventas_cat_cus_mes_all = ventas_cat_cus_mes_all.fillna(0)

# ---------- PASO 2: cálculo de medias, máximos, mínimos y std en ventanas móviles ---------
ventanas = [3, 6, 9, 12]

for cat in categorias:
    col_venta = f'tn_{cat}_cus_mes'
    if col_venta not in ventas_cat_cus_mes_all.columns:
        continue
    ventas_cat_cus_mes_all = ventas_cat_cus_mes_all.sort_values(['customer_id', 'date_mes'])

    for ventana in ventanas:
        ventas_cat_cus_mes_all[f'{col_venta}_mean_{ventana}m'] = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].transform(
            lambda x: x.rolling(window=ventana, min_periods=1).mean()
        )
        ventas_cat_cus_mes_all[f'{col_venta}_max_{ventana}m'] = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].transform(
            lambda x: x.rolling(window=ventana, min_periods=1).max()
        )
        ventas_cat_cus_mes_all[f'{col_venta}_min_{ventana}m'] = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].transform(
            lambda x: x.rolling(window=ventana, min_periods=1).min()
        )
        ventas_cat_cus_mes_all[f'{col_venta}_std_{ventana}m'] = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].transform(
            lambda x: x.rolling(window=ventana, min_periods=1).std()
        )

# ----------- PASO 3: ranking y share en ventas por categoría y mes (entre clientes) -------------

for cat in categorias:
    col_venta = f'tn_{cat}_cus_mes'
    if col_venta not in ventas_cat_cus_mes_all.columns:
        continue

    # Total ventas para esa categoría en cada mes (suma sobre clientes)
    total_mes = ventas_cat_cus_mes_all.groupby('date_mes')[col_venta].transform('sum')
    ventas_cat_cus_mes_all[f'{cat}_total_mes'] = total_mes

    # Ranking por cliente en ese mes (rank 1 = mayor venta)
    ventas_cat_cus_mes_all[f'{cat}_rank_mes'] = ventas_cat_cus_mes_all.groupby('date_mes')[col_venta].rank(
        method='min', ascending=False
    )

    # Share porcentaje ventas cliente respecto total categoría mes
    ventas_cat_cus_mes_all[f'{cat}_share_mes_pct'] = ventas_cat_cus_mes_all[col_venta] / total_mes * 100

# ----------- PASO 4: ranking y share por cliente en categoría (entre meses) -------------

for cat in categorias:
    col_venta = f'tn_{cat}_cus_mes'
    if col_venta not in ventas_cat_cus_mes_all.columns:
        continue

    # Total ventas del cliente en esa categoría (sobre todos los meses)
    total_cus = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].transform('sum')
    ventas_cat_cus_mes_all[f'{cat}_total_cus'] = total_cus

    # Ranking meses para ese cliente (rank 1 = mayor venta)
    ventas_cat_cus_mes_all[f'{cat}_rank_cus'] = ventas_cat_cus_mes_all.groupby('customer_id')[col_venta].rank(
        method='min', ascending=False
    )

    # Share porcentaje ventas mes respecto al total cliente en la categoría
    ventas_cat_cus_mes_all[f'{cat}_share_cus_pct'] = ventas_cat_cus_mes_all[col_venta] / total_cus * 100

# ----------- PASO 5: Unir métricas creadas al DataFrame original -------------

df_cluster_X = df_cluster_X.merge(
    ventas_cat_cus_mes_all,
    on=['customer_id', 'date_mes'],
    how='left'
)

print(df_cluster_X.head())

   customer_id  product_id   date_mes first_customer_date last_customer_date  \
0        10001       20003 2017-01-01          2017-01-01         2019-12-01   
1        10001       20004 2017-01-01          2017-01-01         2019-12-01   
2        10001       20006 2017-01-01          2017-01-01         2019-12-01   
3        10001       20008 2017-01-01          2017-01-01         2019-12-01   
4        10001       20011 2017-01-01          2017-01-01         2019-12-01   

  first_product_date last_product_date       tn  dif_tn  dif_por  vida_prod  \
0         2017-01-01        2019-12-01  100.213   0.000    0.000          4   
1         2017-01-01        2019-12-01   21.740   0.000    0.000          4   
2         2017-01-01        2019-12-01   29.172  -2.196   -0.070          4   
3         2017-01-01        2019-12-01   27.133  -3.450   -0.113          4   
4         2017-01-01        2019-12-01   49.681   0.000    0.000          4   

   vida_customer  meses_on  meses_off  vida_

In [ ]:
df_cluster_X.shape

(174785, 332)

In [ ]:
# lgbm no banca tildes ni espacios en los nombres de columna
# Diccionario de reemplazos para vocales con tilde
replacements = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U',
}

def clean_column_names(col_names):
    cleaned_cols = []
    for col in col_names:
        # Reemplazar espacios por guion bajo
        col = col.replace(' ', '_')
        # Eliminar comas
        col = col.replace(',', '')
        # Reemplazar vocales con tilde
        for orig, repl in replacements.items():
            col = col.replace(orig, repl)
        cleaned_cols.append(col)
    return cleaned_cols

# Aplica al DataFrame df_cluster_X
df_cluster_X.columns = clean_column_names(df_cluster_X.columns)

# Comprobar resultado
print(df_cluster_X.columns)

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por',
       ...
       'cat1_cat2_share_cus_pct', 'cat1_cat2_cat3_total_cus',
       'cat1_cat2_cat3_rank_cus', 'cat1_cat2_cat3_share_cus_pct',
       'cat1_cat2_cat3_brand_total_cus', 'cat1_cat2_cat3_brand_rank_cus',
       'cat1_cat2_cat3_brand_share_cus_pct',
       'cat1_cat2_cat3_brand_size_total_cus',
       'cat1_cat2_cat3_brand_size_rank_cus',
       'cat1_cat2_cat3_brand_size_share_cus_pct'],
      dtype='object', length=332)


In [ ]:
# como el trabajo ha sido interesante guardo el dataset luego tendré solo normalizar y calcular.. solo
df_cluster_X.to_parquet(DATOS_DIR +'df_cluster3_full.parquet', index=False)